In [2]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import Callback
import xgboost as xgb

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 500 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 
            
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [3]:
train_X = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')
train_y = pd.read_json('data/train_interest.json')[0]

In [4]:
cols_to_use = ['BoroCode',
 'Day',
 'Month',
 'NTACode',
 'Wday',
 'Yday',
 'Zero_building_id',
 'Zero_manager_id',
 'address_high_frac',
 'address_low_frac',
 'address_med_frac',
 'address_worths',
 'avenue',
 'bathrooms',
 'bc_price',
 'bedrooms',
 'boro_high_frac',
 'boro_low_frac',
 'boro_med_frac',
 'boro_worths',
 'building_high_frac',
 'building_id',
 'building_low_frac',
 'building_med_frac',
 'building_worths',
 'desc_letters_count',
 'desc_words_count',
 'desc_words_length',
 'display_address',
 'east',
 'features_count',
 'hour',
 'latitude',
 'longitude',
 'manager_high_frac',
 'manager_id',
 'manager_low_frac',
 'manager_med_frac',
 'manager_worths',
 'north',
 'nta_high_frac',
 'nta_low_frac',
 'nta_med_frac',
 'nta_worths',
 'other_address',
 'photos_count',
 'price_by_BoroCode',
 'price_by_NTACode',
 'price_by_area',
 'price_by_building_id',
 'price_by_display_address',
 'price_by_manager_id',
 'price_per_bathroom',
 'price_per_bed_and_bath',
 'price_per_bedroom',
 'real_minus_expected_price',
 'real_over_expected_price',
 'south',
 'sparse_24',
 'sparse_2fullbaths',
 'sparse_actualapt',
 'sparse_airconditioning',
 'sparse_allutilitiesincluded',
 'sparse_assignedparkingspace',
 'sparse_attendedlobby',
 'sparse_backyard',
 'sparse_balconies',
 'sparse_balcony',
 'sparse_basementstorage',
 'sparse_bikeroom',
 'sparse_bikestorage',
 'sparse_billiardsroom',
 'sparse_brownstone',
 'sparse_buildingcommonoutdoorspace',
 'sparse_businesscenter',
 'sparse_cable',
 'sparse_catsallowed',
 'sparse_centrala',
 'sparse_centralac',
 'sparse_centralair',
 'sparse_chef',
 'sparse_chefskitchen',
 'sparse_children',
 'sparse_childrensplayroom',
 'sparse_cityview',
 'sparse_commonbackyard',
 'sparse_commongarden',
 'sparse_commonoutdoorspace',
 'sparse_commonparking',
 'sparse_commonroofdeck',
 'sparse_commonterrace',
 'sparse_communityrecreationfacilities',
 'sparse_concierge',
 'sparse_conciergeservice',
 'sparse_condofinishes',
 'sparse_cook',
 'sparse_courtyard',
 'sparse_deck',
 'sparse_decorativefireplace',
 'sparse_diningroom',
 'sparse_dishwasher',
 'sparse_dogsallowed',
 'sparse_doorman',
 'sparse_drycleaningservice',
 'sparse_dryer',
 'sparse_dryerinbuilding',
 'sparse_dryerinunit',
 'sparse_duplex',
 'sparse_eatinkitchen',
 'sparse_elev',
 'sparse_elevator',
 'sparse_elevbldg',
 'sparse_exclusive',
 'sparse_exposedbrick',
 'sparse_fireplace',
 'sparse_fireplaces',
 'sparse_fitness',
 'sparse_fitnesscenter',
 'sparse_flex2',
 'sparse_flex3',
 'sparse_ftdoorman',
 'sparse_fullservicegarage',
 'sparse_fulltimedoorman',
 'sparse_furnished',
 'sparse_garage',
 'sparse_garden',
 'sparse_gourmetkitchen',
 'sparse_granitecountertops',
 'sparse_granitekitchen',
 'sparse_greenbuilding',
 'sparse_guarantorsaccepted',
 'sparse_gutrenovated',
 'sparse_gym',
 'sparse_gyminbuilding',
 'sparse_hardwood',
 'sparse_hardwoodfloors',
 'sparse_healthclub',
 'sparse_highceiling',
 'sparse_highceilings',
 'sparse_highrise',
 'sparse_highspeedinternet',
 'sparse_hirise',
 'sparse_housekeeping',
 'sparse_indoorpool',
 'sparse_inunitwasher',
 'sparse_largelivingroom',
 'sparse_laundry',
 'sparse_laundryinbuilding',
 'sparse_laundryinunit',
 'sparse_laundryonfloor',
 'sparse_laundryroom',
 'sparse_light',
 'sparse_live',
 'sparse_liveinsuper',
 'sparse_liveinsuperintendent',
 'sparse_lndrybldg',
 'sparse_loft',
 'sparse_lounge',
 'sparse_loungeroom',
 'sparse_lowrise',
 'sparse_luxurybuilding',
 'sparse_mailroom',
 'sparse_marblebath',
 'sparse_marblebathroom',
 'sparse_microwave',
 'sparse_midrise',
 'sparse_mrcleanapproved',
 'sparse_multilevel',
 'sparse_new',
 'sparse_newconstruction',
 'sparse_newlyrenovated',
 'sparse_nofee',
 'sparse_nopets',
 'sparse_nursery',
 'sparse_onsitegarage',
 'sparse_onsitelaundry',
 'sparse_onsiteparking',
 'sparse_onsiteparkingavailable',
 'sparse_onsiteparkinglot',
 'sparse_onsitesuper',
 'sparse_ornateprewardetails',
 'sparse_outdoorareas',
 'sparse_outdoorentertainmentspace',
 'sparse_outdoorpool',
 'sparse_outdoorspace',
 'sparse_packageroom',
 'sparse_parking',
 'sparse_parkingspace',
 'sparse_patio',
 'sparse_penthouse',
 'sparse_petfriendly',
 'sparse_pets',
 'sparse_petsallowed',
 'sparse_petsok',
 'sparse_petsonapproval',
 'sparse_photos',
 'sparse_playroom',
 'sparse_pool',
 'sparse_postwar',
 'sparse_prewar',
 'sparse_privatebackyard',
 'sparse_privatebalcony',
 'sparse_privatedeck',
 'sparse_privatelaundryroomoneveryfloor',
 'sparse_privateoutdoorspace',
 'sparse_privateparking',
 'sparse_privateroofdeck',
 'sparse_privateterrace',
 'sparse_publicoutdoor',
 'sparse_reducedfee',
 'sparse_renovated',
 'sparse_residentsgarden',
 'sparse_residentslounge',
 'sparse_roofaccess',
 'sparse_roofdeck',
 'sparse_rooftopdeck',
 'sparse_rooftopterrace',
 'sparse_roomyclosets',
 'sparse_satellitetv',
 'sparse_sauna',
 'sparse_screeningroom',
 'sparse_sharedbackyard',
 'sparse_sharesok',
 'sparse_shorttermallowed',
 'sparse_simplex',
 'sparse_skitchen',
 'sparse_skylight',
 'sparse_splayroom',
 'sparse_sskitchen',
 'sparse_stainlesssteelappliances',
 'sparse_stepstothepark',
 'sparse_storage',
 'sparse_storagefacilitiesavailable',
 'sparse_storageroom',
 'sparse_sublet',
 'sparse_subway',
 'sparse_sundeck',
 'sparse_swimmingpool',
 'sparse_tenantlounge',
 'sparse_terrace',
 'sparse_terraces',
 'sparse_tonsofnaturallight',
 'sparse_valet',
 'sparse_valetparking',
 'sparse_valetservices',
 'sparse_valetservicesincludingdrycleaning',
 'sparse_videointercom',
 'sparse_view',
 'sparse_virtualdoorman',
 'sparse_walkincloset',
 'sparse_wallsofwindows',
 'sparse_washer',
 'sparse_washerinunit',
 'sparse_wheelchairaccess',
 'sparse_wheelchairramp',
 'sparse_wifi',
 'sparse_wifiaccess',
 'sparse_work',
 'street',
 'west',
 'worth_by_BoroCode',
 'worth_by_NTACode',
 'worth_by_building_id',
 'worth_by_display_address',
 'worth_by_manager_id'] + ['layer1_pca_0', 'layer1_pca_1', 'layer1_pca_2'] + ['layer1_ridge_predictions']

In [5]:
train_X = train_X[cols_to_use]

In [6]:
NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

In [7]:
b = pd.read_json('data/backup/train.json')
a = pd.read_csv('data/listing_image_time.csv')

In [10]:
#train_X['listing_id'] = b.listing_id
#a.columns = ['listing_id', 'time_stamp']
#length = len(train_X)
#train_X = train_X.merge(a, on='listing_id', how='outer')[:length]
train_X.to_json('data/train_final.json', orient='records')

In [11]:
train_X = pd.read_json('data/train_final.json')

In [12]:
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [13]:
cols = list(train_X.columns)
dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
dtest = xgb.DMatrix(data=xte[cols])

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

bst = xgb.train(params, dtrain, best_rounds)
print('train score: ', log_loss(ytr_one_hot, bst.predict(dtrain)))
print('test score: ', log_loss(yte_one_hot, bst.predict(dtest)))

[0]	train-mlogloss:1.09099+1.39914e-05	test-mlogloss:1.09119+3.67423e-05
[200]	train-mlogloss:0.569646+0.000608634	test-mlogloss:0.601149+0.00177252
[400]	train-mlogloss:0.488264+0.000712651	test-mlogloss:0.543924+0.00235677
[600]	train-mlogloss:0.451513+0.000902248	test-mlogloss:0.529166+0.00230609
[800]	train-mlogloss:0.424251+0.000696344	test-mlogloss:0.52251+0.00217816
[1000]	train-mlogloss:0.401281+0.000692514	test-mlogloss:0.518659+0.00206709
[1200]	train-mlogloss:0.380412+0.000862049	test-mlogloss:0.516107+0.00187526
[1400]	train-mlogloss:0.361306+0.000611157	test-mlogloss:0.514412+0.0017931
[1600]	train-mlogloss:0.343518+0.000660291	test-mlogloss:0.513296+0.00175838
[1800]	train-mlogloss:0.3269+0.00067361	test-mlogloss:0.512436+0.00175134
[2000]	train-mlogloss:0.31124+0.000572575	test-mlogloss:0.511954+0.00178174
[2200]	train-mlogloss:0.296558+0.000678825	test-mlogloss:0.511569+0.00176298
0.5115074
train score:  0.318907893692
test score:  0.532676951063


In [14]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers.core import ActivityRegularization
def create_model(xtr, layer1=256, layer2=32, activation='tanh', activation2=PReLU, dropout=0.75):
    model = Sequential()
    
    model.add(Dense(layer1,
                    input_dim=xtr.shape[1],
                    init = 'he_normal',
                    activation=activation))
                    #kernel_regularizer=regularizers.l2(0.01),
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(layer2,
                    init = 'he_normal',
                    activation=activation))
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    model.add(BatchNormalization())    
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [ ]:
%%time
filepath = 'data/nn_weights'

nn_cols = [score[0] for score in scores if score[1] > 200] #list(train_X.columns)

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr[nn_cols])
model.fit(xtr[nn_cols].values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte[nn_cols].values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])

model = create_model(xtr[nn_cols])
model.load_weights(filepath)
nn_preds = model.predict(xte[nn_cols].values)
print(log_loss(yte_one_hot, nn_preds))

In [28]:
%%time
et_cols = nn_cols

test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=25,
    max_depth=10
    )

test.fit(xtr[et_cols], ytr)
            
et_train_predictions = test.predict_proba(xtr[et_cols])
et_predictions = test.predict_proba(xte[et_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, et_train_predictions)))
# 0.5824240849016719 (test loss 0.3909099642627144)

0.6284038621636637 (test loss 0.6043116367972)
CPU times: user 5.69 s, sys: 201 ms, total: 5.89 s
Wall time: 5.99 s


In [29]:
%%time
rf_cols = nn_cols

test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=25,
    max_depth=10)

test.fit(xtr[rf_cols], ytr)
            
rf_train_predictions = test.predict_proba(xtr[rf_cols])
rf_predictions = test.predict_proba(xte[rf_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, rf_train_predictions)))
# 0.5714659966135285 (test loss 0.37359374722034255)

0.5823083608180835 (test loss 0.5111770718368133)
CPU times: user 12.1 s, sys: 197 ms, total: 12.3 s
Wall time: 12.6 s


In [18]:
%%time
knn_features = nn_cols

test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[knn_features], ytr)

knn_train_predictions = test.predict_proba(xtr[knn_features])
knn_predictions = test.predict_proba(xte[knn_features])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6327628575671347 (test loss 0.6255989023665038)
CPU times: user 16min 7s, sys: 2.28 s, total: 16min 10s
Wall time: 16min 11s


In [19]:
score_map = bst.get_fscore()
scores = [(key, score_map[key]) for key in score_map]

In [20]:
sorted(scores, key=lambda x: x[1])

[('sparse_onsitegarage', 1),
 ('sparse_gutrenovated', 1),
 ('sparse_privateroofdeck', 1),
 ('sparse_screeningroom', 1),
 ('sparse_new', 1),
 ('sparse_roofaccess', 1),
 ('sparse_videointercom', 1),
 ('sparse_privatebackyard', 1),
 ('sparse_parking', 1),
 ('sparse_wifi', 1),
 ('sparse_pets', 1),
 ('sparse_centralac', 1),
 ('sparse_petsok', 1),
 ('sparse_2fullbaths', 1),
 ('sparse_gourmetkitchen', 1),
 ('sparse_dryerinbuilding', 1),
 ('sparse_sharesok', 1),
 ('sparse_satellitetv', 1),
 ('sparse_largelivingroom', 1),
 ('sparse_indoorpool', 2),
 ('sparse_laundryonfloor', 2),
 ('sparse_skitchen', 2),
 ('sparse_buildingcommonoutdoorspace', 2),
 ('sparse_outdoorpool', 2),
 ('sparse_postwar', 2),
 ('sparse_flex3', 2),
 ('sparse_rooftopterrace', 2),
 ('sparse_live', 2),
 ('sparse_petsallowed', 2),
 ('sparse_valet', 3),
 ('sparse_courtyard', 3),
 ('sparse_storageroom', 3),
 ('sparse_petfriendly', 3),
 ('sparse_drycleaningservice', 3),
 ('sparse_cable', 3),
 ('sparse_subway', 3),
 ('sparse_microwa

In [ ]:
cols = [score[0] for score in scores if score[1] > 500]

In [31]:
len(cols)

89

In [32]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[cols], ytr)
knn_train_predictions = test.predict_proba(xtr[cols])
knn_predictions = test.predict_proba(xte[cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))
#0.6220178408396022

0.6286478216063587 (test loss 0.6182119627630555)
CPU times: user 9min 15s, sys: 3.1 s, total: 9min 18s
Wall time: 9min 27s


In [ ]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[cols], ytr)
knn_train_predictions = test.predict_proba(xtr[cols])
knn_predictions = test.predict_proba(xte[cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))
#0.6242524229135699

0.6253617197489705 (test loss 0.6059399788946489)
CPU times: user 8min 46s, sys: 1.33 s, total: 8min 48s
Wall time: 8min 49s


In [26]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[cols], ytr)
knn_train_predictions = test.predict_proba(xtr[cols])
knn_predictions = test.predict_proba(xte[cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6228269282266731 (test loss 0.6057507141706729)
CPU times: user 6min 6s, sys: 1.18 s, total: 6min 7s
Wall time: 6min 7s


In [27]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[cols], ytr)
knn_train_predictions = test.predict_proba(xtr[cols])
knn_predictions = test.predict_proba(xte[cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))

0.6283323043737444 (test loss 0.5945800788494582)
CPU times: user 5min 52s, sys: 712 ms, total: 5min 53s
Wall time: 5min 53s
